# PART 1 

Scrape the following Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
Create the above dataframe:
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area, combine them into one row with the - neighborhoods separated with a comma.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
# import libraries
!pip install BeautifulSoup4
!pip install lxml
import requests
from bs4 import BeautifulSoup
import pandas as pd
print('Libraries Done')

     |████████████████████████████████| 122kB 7.0MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 6.1MB/s eta 0:00:01
Libraries Done


## Scrapping ##

Scrape the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [6]:
# import URL
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [7]:
soup = BeautifulSoup(source, 'html')
table=soup.find('table')


* Create DataFrame with 3 columns

In [8]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

* Populate DataFrame

In [9]:
#tr contains the postalcode
for row in table.find_all('tr'):
    row_data=[]
    for cell in row.find_all('td'):
        row_data.append(cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [10]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
df.shape

(180, 3)

In [12]:
df.groupby(['Borough'])['Neighborhood'].count()              # count anighborhood by Borough

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Not assigned        77
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

## Data Cleaning

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.= 77 rows

In [13]:
df=df[df['Borough']!='Not assigned']

In [14]:
df.shape

(103, 3)

*  Neighborhoods with same Postalcode would be combine into one row

In [15]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)


In [16]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [17]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [18]:
df_merge.drop_duplicates(inplace=True)

In [19]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [20]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [21]:
df_merge.shape

(103, 3)

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [23]:
df_merge.loc[df_merge['Neighborhood'] == 'Not assigned', ['Neighborhood']] = df_merge['Borough']
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
# total postalcodes by Borough 
df_merge.groupby(['Borough'])['Postalcode'].count()


Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Postalcode, dtype: int64

In [25]:
df_merge.to_csv('df_cleaned.csv')                       # save Dataset for Next Assignment

* Final result

In [26]:
df_merge.shape

(103, 3)